In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [ ]:
main_link = 'https://hh.ru'

search_vacancy = input('Введите вакансию для поиска: ')
page = 0
while True:
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

    params = {'text': search_vacancy,
              'clusters': 'true',
              'enable_snippets': 'true',
              'salary': '',
              'st': 'searchVacancy',
              'fromSearch': 'true'}

    response = requests.get(main_link + '/search/vacancy/', params=params, headers=header)
    soup = bs(response.text, 'html.parser')

    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})

    vacancy_list = vacancy_block.findChildren(recursive=False)

    vacancies = []

    for vacancy in vacancy_list:
        if vacancy['class'] == ['vacancy-serp-item'] or vacancy['class'] == ['vacancy-serp-item', 'vacancy-serp-item_premium']:
            link = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
            company = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).text
            city = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).text
            name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
            try:
                salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            except AttributeError:
                salary = 'None'
            salary = salary.replace('\xa0', '')

            vacancy_data = {'link': link,
                            'company': company,
                            'city': city,
                            'name': name,
                            'salary': salary}

            vacancies.append(vacancy_data)

    page += 1

    if not soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}):
        break


In [ ]:
df_vacancies = pd.DataFrame(vacancies)

In [ ]:
df_vacancies.head(5)